# Analisis dan Prediksi pada Data Audio Emosi


Nama: Whinta Virginia Putri

NIM: 210411100047

UTS Proyek Sain Data

In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import joblib
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import requests
from io import StringIO
from sklearn.preprocessing import StandardScaler
import pickle
from joblib import dump
from sklearn.decomposition import PCA as sklearnPCA
import math
from google.colab import drive
import os
from google.colab import files

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## Deskripsi Dataset :

 Ada sekitar 200 kata target yang diucapkan dalam frase pembawa 'say the word_' oleh dua aktris (berusia 26 dan 64 tahun) dan rekaman dibuat dari set tersebut menggambarkan tujuh emosi (marah, jijik, takut, bahagia, terkejut menyenangkan, sedih, dan netral). Terdapat total 2800 titik data (file audio).

 Kumpulan data ini diatur sedemikian rupa sehingga setiap dari dua aktris perempuan dan emosi mereka terdapat dalam folder tersendiri. Dan di dalamnya, semua file audio 200 kata target dapat ditemukan. Format file audio ini adalah format WAV.

 Berikut link kaggle dataset: https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess?resource=download


## Pre-processing


  Dibuat 11 fitur dari Dataset audio kaggle yaitu Label, Mean, Median, Std_Deviation, Zero_Crossing_Rate, Energy, Spectral_Centroid, Spectral_Bandwidth, Spectral_Rolloff, Chroma, dan MFCCs dari 2191 data, Kemudian di simpan pada format csv.
  Berikut adalah hasil Audio_features:

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/whintaaa/datapsd/main/audio_fitur.csv')
df

,Label,Audio_Name,Mean,Median,Std_Deviation,Zero_Crossing_Rate,Energy,Spectral_Centroid,Spectral_Bandwidth,Spectral_Rolloff,Chroma,MFCCs
0,disgust,YAF_chief_disgust,-0.000022,-0.000181,0.022747,0.232713,0.017588,3447.960816,2174.962231,5621.252553,0.391336,-30.599363
1,disgust,YAF_chain_disgust,-0.000018,-0.000350,0.020867,0.207239,0.015964,3139.689603,2052.783771,4998.484497,0.342745,-30.325106
2,disgust,YAF_base_disgust,-0.000021,-0.000203,0.022053,0.246661,0.017039,3609.513207,2183.848242,5606.822791,0.348650,-30.605677
3,disgust,YAF_ditch_disgust,-0.000024,-0.000103,0.020262,0.196533,0.015652,3202.293486,2061.560496,5041.901634,0.365224,-30.454414
4,disgust,YAF_germ_disgust,-0.000018,-0.000353,0.025729,0.192057,0.020146,2898.874228,1902.174869,4483.722888,0.321074,-30.068130
...,...,...,...,...,...,...,...,...,...,...,...,...
2185,Sad,OAF_wheat_sad,0.000033,0.000606,0.010872,0.086951,0.008632,1816.902526,2151.488369,3230.488327,0.252862,-32.830270
2186,Sad,OAF_puff_sad,-0.000010,0.000292,0.013971,0.078135,0.011377,1743.116982,2153.268985,3179.314108,0.375577,-32.812740
2187,Sad,OAF_raise_sad,-0.000001,0.000284,0.013523,0.086271,0.011093,1922.328544,2198.764520,3393.786621,0.297995,-33.027540
2188,Sad,OAF_sail_sad,0.000032,0.000330,0.010211,0.118340,0.008695,2245.346145,2210.531313,3846.132299,0.257481,-32.356060


In [5]:
# Memisahkan fitur (X) dan label (y)
X = df.drop(['Label','Audio_Name'], axis=1)
y = df['Label']
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2)


### Normalisasi Scaler

Normalisasi scaler dalam konteks kode yang Anda berikan tampaknya menggunakan `StandardScaler` dari pustaka scikit-learn untuk melakukan normalisasi pada data. Normalisasi adalah proses mengubah nilai-nilai dalam dataset ke skala umum, biasanya dengan mengurangkan rata-rata dan membagi hasilnya dengan deviasi standar.

Mari kita bahas langkah-langkah dalam kode tersebut:

1. **Define and Fit Scaler:**
   ```python
   scaler = StandardScaler()
   scaler.fit(X_train)
   ```
   Pada langkah ini, Anda membuat objek `StandardScaler` dari scikit-learn dan kemudian menggunakan metode `fit` untuk menghitung rata-rata dan deviasi standar dari dataset pelatihan (`X_train`).

2. **Save Scaler Using Pickle:**
   ```python
   scaler_file_path = r'/content/drive/My Drive/prosaindata/tugas/scaler.pkl'
   with open(scaler_file_path, 'wb') as scaler_file:
       pickle.dump(scaler, scaler_file)
   ```
   Pada langkah ini, Anda menyimpan objek scaler ke dalam file menggunakan modul `pickle`. Ini memungkinkan Anda untuk menggunakan kembali objek scaler tanpa harus melatih ulang model setiap kali Anda menjalankan program.

3. **Transform Training Data Using Scaler:**
   ```python
   X_train_scaled = scaler.transform(X_train)
   ```
   Di sini, Anda menggunakan scaler yang telah dilatih untuk melakukan normalisasi pada dataset pelatihan (`X_train`) dan menyimpan hasilnya dalam `X_train_scaled`.

4. **Load Scaler Using Pickle:**
   ```python
   with open(r'/content/drive/My Drive/prosaindata/tugas/scaler.pkl', 'rb') as normalisasi:
       loadscal = pickle.load(normalisasi)
   ```
   Langkah terakhir adalah memuat kembali objek scaler dari file yang telah disimpan sebelumnya menggunakan modul `pickle`. Objek scaler yang dimuat disimpan dalam variabel `loadscal`.

Dengan cara ini, Anda dapat menggunakan objek `loadscal` untuk melakukan transformasi normalisasi pada dataset lain dengan menggunakan parameter normalisasi yang sama seperti yang telah dihitung pada dataset pelatihan.

In [6]:
# Define and fit the scaler on the training dataset
scaler = StandardScaler()
scaler.fit(X_train)
# Save the scaler using pickle
scaler_file_path = r'/content/drive/My Drive/prosaindata/tugas/scaler.pkl'
with open(scaler_file_path, 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

X_train_scaled = scaler.transform(X_train)
with open(r'/content/drive/My Drive/prosaindata/tugas/scaler.pkl', 'rb') as normalisasi:
    loadscal = pickle.load(normalisasi)


In [7]:
X_test_scaled = loadscal.transform(X_test)

In [8]:
X_test_scaled

array([[-0.36438935, -0.01864303,  0.62951586, ...,  1.700305  ,
         1.54756458,  0.50833751],
       [-0.4145327 , -0.16604988,  0.48640237, ...,  0.96098207,
        -0.94090678,  0.60164653],
       [-0.54970173, -0.37708373,  0.00366902, ...,  0.23587569,
        -0.06940957,  0.06954893],
       ...,
       [-0.47993707, -0.22850376, -0.05819755, ...,  0.90422257,
         0.30463648, -0.36001778],
       [ 0.80198861, -0.0505326 , -1.00057672, ...,  0.02687033,
        -0.90359502, -1.22195475],
       [-0.63036712,  0.86737544, -0.17119103, ...,  0.01166204,
        -0.94401292,  1.46828321]])

## Modelling

### Mencari k terbaik dengan akurasi paling tinggi

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
K = 30
acc = np.zeros((K - 1))

for n in range(1, K, 2):
    knn = KNeighborsClassifier(n_neighbors=n, metric="euclidean").fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)
    acc[n - 1] = accuracy_score(y_test, y_pred)

best_accuracy = acc.max()
best_k = acc.argmax() + 1

# Tampilkan akurasi terbaik dan nilai k
print('Akurasi terbaik adalah', best_accuracy, 'dengan nilai k =', best_k)


Akurasi terbaik adalah 0.769406392694064 dengan nilai k = 9


### Menyimpan model knn dengan k terbaik

In [10]:
# Simpan model KNN terbaik
best_knn = KNeighborsClassifier(n_neighbors=best_k, metric="euclidean")
best_knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=9)

In [11]:
# Save the best KNN model using pickle
model_file_path = r'/content/drive/My Drive/prosaindata/tugas/model.pkl'
with open(model_file_path, 'wb') as model_file:
    pickle.dump(best_knn, model_file)

with open(r'/content/drive/My Drive/prosaindata/tugas/model.pkl', 'rb') as knn_model:
    load_knn = pickle.load(knn_model)


In [12]:
y_pred = load_knn.predict(X_test_scaled)
y_pred

array(['angry', 'fear', 'pleasantsurprised', 'Sad', 'neutral', 'disgust',
       'pleasantsurprised', 'neutral', 'happy', 'Fear', 'Fear', 'angry',
       'pleasantsurprised', 'disgust', 'happy', 'pleasantsurprised',
       'sad', 'neutral', 'sad', 'happy', 'happy', 'disgust', 'Fear',
       'Sad', 'angry', 'sad', 'sad', 'neutral', 'disgust', 'disgust',
       'angry', 'pleasantsurprised', 'neutral', 'sad', 'Sad',
       'pleasantsurprised', 'Fear', 'disgust', 'Fear', 'happy', 'happy',
       'neutral', 'happy', 'Fear', 'Pleasantsurprise', 'neutral', 'Fear',
       'disgust', 'disgust', 'sad', 'pleasantsurprised', 'neutral',
       'happy', 'neutral', 'neutral', 'disgust', 'Fear',
       'pleasantsurprised', 'sad', 'happy', 'Pleasantsurprise', 'neutral',
       'Fear', 'Sad', 'pleasantsurprised', 'disgust', 'Sad',
       'pleasantsurprised', 'disgust', 'Fear', 'Sad', 'Pleasantsurprise',
       'disgust', 'sad', 'Sad', 'angry', 'Sad', 'happy', 'angry',
       'neutral', 'sad', 'disgust',

In [13]:
print('Akurasi KNN dengan data test:')
accuracy = accuracy_score(y_test, y_pred)
print( accuracy)

Akurasi KNN dengan data test:
0.769406392694064


In [14]:
# Hitung prediksi label KNN
knn_predictions = load_knn.predict(X_test_scaled)

# Simpan hasil prediksi KNN ke dalam DataFrame
knn_results_df = pd.DataFrame({'Actual Label': y_test, 'Predicted Label (KNN)': knn_predictions})

# Tampilkan tabel prediksi KNN
print("Tabel Prediksi Label KNN")
knn_results_df

Tabel Prediksi Label KNN


,Actual Label,Predicted Label (KNN)
353,angry,angry
1631,fear,fear
414,pleasantsurprised,pleasantsurprised
1827,neutral,Sad
851,neutral,neutral
...,...,...
1613,fear,fear
50,disgust,disgust
453,pleasantsurprised,pleasantsurprised
1822,neutral,Sad


### Reduksi PCA

Reduksi PCA (Principal Component Analysis) adalah teknik yang digunakan untuk mengurangi dimensi dari dataset yang kompleks, dengan tetap mempertahankan sebagian besar informasi yang terkandung dalam dataset tersebut. PCA bekerja dengan mentransformasi data asli ke dalam ruang fitur baru yang disebut komponen utama atau principal components. Komponen utama ini diurutkan berdasarkan seberapa besar variansinya, sehingga komponen pertama menyimpan sebagian besar varians, yang diikuti oleh komponen kedua, dan seterusnya.



In [15]:
# Lakukan reduksi PCA
sklearn_pca = sklearnPCA(n_components=10)
X_train_pca = sklearn_pca.fit_transform(X_train_scaled)

print("Principal Components 10:")
X_train_pca


Principal Components 10:


array([[-2.15699059,  0.12088686, -1.05816054, ...,  0.15522723,
        -0.09174985, -0.01146861],
       [ 2.70852347, -0.64630269,  0.05224175, ...,  0.00458544,
        -0.04659703, -0.08161985],
       [ 2.91065674,  0.09092699, -0.62907659, ...,  0.05744571,
        -0.00876878, -0.01138743],
       ...,
       [ 1.00230967, -0.72522836,  2.26887371, ..., -0.10442859,
         0.15861958, -0.09644085],
       [ 3.10754228, -0.69602145,  0.38281969, ...,  0.05710673,
         0.06719287,  0.07069764],
       [-2.00722733,  0.50821046,  1.9109502 , ..., -0.00369189,
        -0.05388441,  0.01722397]])

In [16]:
# Save the PCA model
pca_model_file_path = r'/content/drive/My Drive/prosaindata/tugas/PCA10.pkl'
with open(pca_model_file_path, 'wb') as pca_model_file:
    pickle.dump(sklearn_pca, pca_model_file)
# Load the PCA model
with open(pca_model_file_path, 'rb') as pca_model:
    loadpca = pickle.load(pca_model)


In [17]:
# Transform test data using the loaded PCA model
X_test_pca = loadpca.transform(X_test_scaled)

# Continue with KNN and evaluation as needed
K = 30
acc_pca = np.zeros((K - 1))
for n in range(1, K, 2):
    knn_pca = KNeighborsClassifier(n_neighbors=n, metric="euclidean").fit(X_train_pca, y_train)
    y_pred_pca = knn_pca.predict(X_test_pca)
    acc_pca[n - 1] = accuracy_score(y_test, y_pred_pca)

best_accuracy_pca = acc_pca.max()
best_k_pca = acc_pca.argmax() + 1

# Tampilkan akurasi terbaik dan nilai k dengan PCA
print('Akurasi KNN terbaik dengan PCA adalah', best_accuracy_pca, 'dengan nilai k =', best_k_pca)

Akurasi KNN terbaik dengan PCA adalah 0.769406392694064 dengan nilai k = 9


In [18]:
# Hitung prediksi label KNN setelah PCA
knn_pca_predictions = knn_pca.predict(X_test_pca)

# Simpan hasil prediksi KNN setelah PCA ke dalam DataFrame
knn_pca_results_df = pd.DataFrame({'Actual Label': y_test, 'Predicted Label (KNN with PCA)': knn_pca_predictions})

# Tampilkan tabel prediksi KNN setelah PCA
print("Tabel Prediksi Label KNN dengan PCA")
knn_pca_results_df

Tabel Prediksi Label KNN dengan PCA


,Actual Label,Predicted Label (KNN with PCA)
353,angry,angry
1631,fear,fear
414,pleasantsurprised,neutral
1827,neutral,Sad
851,neutral,neutral
...,...,...
1613,fear,fear
50,disgust,disgust
453,pleasantsurprised,pleasantsurprised
1822,neutral,neutral


## Evaluasi

### Prediksi Data Audio

In [19]:
import librosa
import numpy as np

# Fungsi untuk menghitung fitur audio
def extract_features(audio_path):
    y, sr = librosa.load(audio_path)

    # Fitur 1: Mean
    mean = np.mean(y)

    # Fitur 2: Median
    median = np.median(y)

    # Fitur 3: Standard Deviation
    std_deviation = np.std(y)

    # Fitur 4: Zero Crossing Rate
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))

    # Fitur 5: Energy
    energy = np.mean(librosa.feature.rms(y=y))

    # Fitur 6: Spectral Centroid
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))

    # Fitur 7: Spectral Bandwidth
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))

    # Fitur 8: Spectral Roll-off
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))

    # Fitur 9: Chroma Feature
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))

    # Fitur 10: Mel-frequency Cepstral Coefficients (MFCCs)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13))

    return [mean, median, std_deviation, zero_crossing_rate, energy,
            spectral_centroid, spectral_bandwidth, spectral_rolloff, chroma, mfccs]

print('Ekstraksi Fitur Audio')
print('Unggah file audio WAV untuk menghitung fitur statistiknya.')

# Unggah file audio
from google.colab import files
uploaded_audio = files.upload()

if uploaded_audio:
    audio_path = list(uploaded_audio.keys())[0]
    print(f"Audio file: {audio_path}")

    audio_features = extract_features(audio_path)

    feature_names = [
        "Mean", "Median", "Std Deviation", "Zero Crossing Rate", "Energy",
        "Spectral Centroid", "Spectral Bandwidth", "Spectral Rolloff", "Chroma", "MFCCs"
    ]

    # Tampilkan hasil fitur
    print("### Hasil Ekstraksi Fitur Audio:")
    for i, feature in enumerate(audio_features):
        print(f"{feature_names[i]}: {feature}")

    # Transform audio_features using the loaded scaler
    # (Pastikan loadscal, loadpca, dan load_knn sudah di-load sebelumnya)
    datauji = loadscal.transform(np.array(audio_features).reshape(1, -1))
    datapca = loadpca.transform(datauji)
    # Make predictions using the KNN model
    y_pred_uji = load_knn.predict(datauji)
    # y_pred_pca = load_knn.predict(datapca)

    print("Fitur-fitur setelah di normalisasi: ", datauji)
    print("Data PCA:", datapca)
    # print("Predicted Label (PCA):", y_pred_pca)
    print("Predicted Label (KNN):", y_pred_uji)


Ekstraksi Fitur Audio
Unggah file audio WAV untuk menghitung fitur statistiknya.


Saving OAF_bite_disgust.wav to OAF_bite_disgust.wav
Audio file: OAF_bite_disgust.wav
### Hasil Ekstraksi Fitur Audio:
Mean: -2.2098407498560846e-05
Median: 0.000759080343414098
Std Deviation: 0.02309066243469715
Zero Crossing Rate: 0.10909799429086539
Energy: 0.01691955141723156
Spectral Centroid: 2123.0714181321196
Spectral Bandwidth: 2208.9362323255236
Spectral Rolloff: 3738.598867563101
Chroma: 0.36925455927848816
MFCCs: -29.30312156677246
Fitur-fitur setelah di normalisasi:  [[-0.09837685  0.32714865 -0.59620418 -0.77077837 -0.70190759 -0.85917477
   0.08174241 -0.77696305  1.57644537 -0.13066898]]
Data PCA: [[-1.48356266 -0.39089473 -0.6603022  -0.33854531  1.50702673 -0.25807244
   0.38986447  0.08743024  0.08100064  0.0377777 ]]
Predicted Label (KNN): ['disgust']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## Deployment

Link Streamlit/Aplikasi Prediksi pada data audio emosi:
[https://psd-audio-emosi.streamlit.app](https://psd-audio-emosi.streamlit.app/)
